In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#데이터를 불러와

import numpy as np

rows = np.loadtxt("./lotto.csv", delimiter=",")
row_count = len(rows)
print(row_count)

#이번주 당첨번호
lucky_numbers=[12,20,26,33,44,45]
second_number=[24]

991


In [3]:
import numpy as np

# 당첨번호를 원핫인코딩벡터(ohbin)으로 변환
def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

# 원핫인코딩벡터(ohbin)를 번호로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers

In [4]:
numbers = rows[:, 1:7]
ohbins = list(map(numbers2ohbin, numbers))

x_samples = ohbins[0:row_count-1]
y_samples = ohbins[1:row_count]

#원핫인코딩으로 표시
print("ohbins")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

#번호로 표시
print("numbers")
print("X[0]: " + str(ohbin2numbers(x_samples[0])))
print("Y[0]: " + str(ohbin2numbers(y_samples[0])))

ohbins
X[0]: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
Y[0]: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
numbers
X[0]: [10, 23, 29, 33, 37, 40]
Y[0]: [9, 13, 21, 25, 32, 42]


In [5]:
#windows를 만들자

seq_len = 30
sequence_length = seq_len + 1

result=[]
for index in range(len(x_samples)-sequence_length):
    result.append(x_samples[index: index+sequence_length])

    

In [6]:
result = np.array(x_samples)

row = int(round(result.shape[0]*0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train,(x_train.shape[0], x_train.shape[1],-1))
y_train = train[:,-1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],-1))
y_test = result[row:,-1]

x_train.shape, x_test.shape


((891, 44, 1), (99, 44, 1))

In [97]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

#from keras.layers import Dense, Activation 
from keras.layers import LeakyReLU

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda

from sklearn.metrics import f1_score 
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

optimizer = Adam(0.0051)
loss = Huber()

#모델을 컴파일 해보자
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(50,1)))
model.add(LSTM(64, return_sequences=False))
#model.add(Dense(128, activation='relu'))
model.add(Dense(128))
model.add(Dense(45, activation='sigmoid'))
model.add(LeakyReLU(alpha=0.01))
#model.compile(loss=loss, optimizer=optimizer, metrics=f1_m)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=f1_m)

model.summary()


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 50, 50)            10400     
_________________________________________________________________
lstm_27 (LSTM)               (None, 64)                29440     
_________________________________________________________________
dense_27 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_28 (Dense)             (None, 45)                5805      
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 45)                0         
Total params: 53,965
Trainable params: 53,965
Non-trainable params: 0
_________________________________________________________________


In [98]:
#Training

model.fit(x_train, y_train,
         validation_data=(x_test, y_test),
         batch_size=20,
         epochs=50,
         callbacks = [tf.keras.callbacks.EarlyStopping(patience=15, monitor ='val_loss')])

Epoch 1/50
45/45 [==============================] - 9s 69ms/step - loss: 0.4533 - f1_m: 0.0289 - val_loss: 0.3274 - val_f1_m: 0.0000e+00
Epoch 2/50
45/45 [==============================] - 2s 48ms/step - loss: 0.4195 - f1_m: 0.0000e+00 - val_loss: 0.2933 - val_f1_m: 0.0000e+00
Epoch 3/50
45/45 [==============================] - 2s 49ms/step - loss: 0.4216 - f1_m: 0.0000e+00 - val_loss: 0.2878 - val_f1_m: 0.0000e+00
Epoch 4/50
45/45 [==============================] - 2s 47ms/step - loss: 0.4131 - f1_m: 0.0000e+00 - val_loss: 0.2980 - val_f1_m: 0.0000e+00
Epoch 5/50
45/45 [==============================] - 2s 47ms/step - loss: 0.4215 - f1_m: 0.0000e+00 - val_loss: 0.2820 - val_f1_m: 0.0000e+00
Epoch 6/50
45/45 [==============================] - 2s 49ms/step - loss: 0.4186 - f1_m: 0.0000e+00 - val_loss: 0.2956 - val_f1_m: 0.0000e+00
Epoch 7/50
45/45 [==============================] - 2s 48ms/step - loss: 0.4187 - f1_m: 0.0000e+00 - val_loss: 0.3095 - val_f1_m: 0.0000e+00
Epoch 8/50
45/45 

In [99]:
def gen_numbers_from_probability(nums_prob, exp_numb, inv_numb):

    ball_box = []#꺼낼 공 모음
    selected_balls = []#꺼낸 공 모음


#제외수는 빼
    for i in range (len(exp_numb)):
        nums_prob[exp_numb[i]-1] = 0

#역수를 찾아서 하나는 넣을거야
    inv_ran = np.random.randint(len(inv_numb), size=1)[0]#랜덤번호 하나 뽑아
    inv_ball = inv_numb[inv_ran]#랜덤번호로 역수 리스트에서 하나 수 뽑아
    selected_balls.append(inv_ball)#그 수를 꺼낸공 모음에 넣어
    
#나온 확률에 100을 곱해서 공을 만들고 그중에 뽑는거

    for n in range(45):
        ball_count = int(nums_prob[n] * 10000)#확률 제로면 안뽑을거야
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    
#6개 뽑으면 끝남
    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    return selected_balls



''' 얘네가 원래 꺼

    ball_box = []
    


    for i in range (len(exp_numb)):
        nums_prob[exp_numb[i]-1] = 0
    
    for n in range(45):
        ball_count = int(nums_prob[n] * 100)
        #ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    return selected_balls

'''

' 얘네가 원래 꺼\n\n    ball_box = []\n    \n\n\n    for i in range (len(exp_numb)):\n        nums_prob[exp_numb[i]-1] = 0\n    \n    for n in range(45):\n        ball_count = int(nums_prob[n] * 100)\n        #ball_count = int(nums_prob[n] * 100 + 1)\n        ball = np.full((ball_count), n+1) #1부터 시작\n        ball_box += list(ball)\n\n    selected_balls = []\n\n    while True:\n        \n        if len(selected_balls) == 6:\n            break\n        \n        ball_index = np.random.randint(len(ball_box), size=1)[0]\n        ball = ball_box[ball_index]\n\n        if ball not in selected_balls:\n            selected_balls.append(ball)\n\n    return selected_balls\n\n'

In [100]:
# 마지막 회차까지 학습한 모델로 다음 회차 추론
# 지난 회차 역수 구해(inverse_numbers)

print('receive numbers')


xs = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],-1))

ys_pred = model.predict_on_batch(xs)


list_numbers = []
inverse_numbers = []

for i in range(6):
    a = 46 - ohbin2numbers(y_samples[-1])[i]
    inverse_numbers.append(a)
inverse_numbers.sort()

#여기부터 제외수 만들기
expect_numbers=[]
'''
#평균*0.9 보다 작은 값은 제외수로 넣어
_sum = 0
_mean = 0
for x in ys_pred[0]:
    if x <= 0:
        x = 0
    else:
        _sum += x
_mean = _sum / len(ys_pred[0])

_expect = []
'''

temp1 = list(set(ohbin2numbers(y_samples[-1])) & set(ohbin2numbers(y_samples[-2])))
expect_numbers.extend(temp1)
expect_numbers.append(int(rows[-1,7] + 1))

_mean = np.mean(ys_pred[0])
_std = np.std(ys_pred[0])


for i in range(45):
    if ys_pred[0][i] <= (_mean - _std * 1.1):
        expect_numbers.append(i+1)

'''
#0보다 작은 값은 제외수로 넣어
for j in range(45):
    if ys_pred[0][j] <= 0:
        expect_numbers.append(j+1)
        ys_pred[0][j] = 0
'''

n=0
while n < 100:
#for n in range(100):
    numbers = gen_numbers_from_probability(ys_pred[0],expect_numbers, inverse_numbers)
    numbers.sort()
    if (len(list(set(numbers).intersection(ohbin2numbers(y_samples[-1])))) == 1) and (len(list(set(numbers).intersection(inverse_numbers))) ==1):
        if len(list(set(numbers).intersection(lucky_numbers))) == 3: 
            print('{0} : {1} --- 5등 당첨'.format(n, numbers))
        elif len(list(set(numbers).intersection(lucky_numbers))) == 4: 
            print('{0} : {1} --- 4등 당첨'.format(n, numbers))
        elif len(list(set(numbers).intersection(lucky_numbers))) == 5: 
            print('{0} : {1} --- 3등 당첨'.format(n, numbers))
        elif len(list(set(numbers).intersection(lucky_numbers))) == 6: 
            print('{0} : {1} --- 1등 당첨'.format(n, numbers))
        elif len(list(set(numbers).intersection(lucky_numbers))) == 5 and len(list(set(numbers).intersection(second_number))) == 1: 
            print('{0} : {1} --- 2등 당첨'.format(n, numbers))
        
        else:
            print('{0} : {1}'.format(n, numbers))
        n = n + 1
    list_numbers.append(numbers)

receive numbers
0 : [13, 19, 29, 37, 41, 43]
1 : [3, 6, 21, 22, 31, 41]
2 : [7, 17, 26, 29, 30, 33]
3 : [4, 16, 23, 26, 28, 31]
4 : [3, 22, 26, 32, 33, 35]
5 : [4, 9, 12, 17, 19, 33]
6 : [5, 8, 28, 32, 37, 44]
7 : [1, 21, 23, 34, 41, 44]
8 : [6, 20, 22, 27, 28, 44]
9 : [4, 8, 13, 17, 34, 43]
10 : [1, 10, 27, 33, 35, 43]
11 : [1, 3, 5, 9, 13, 23]
12 : [3, 5, 22, 32, 33, 41]
13 : [5, 12, 28, 30, 31, 42]
14 : [1, 5, 12, 13, 34, 38]
15 : [7, 12, 16, 33, 35, 42]
16 : [1, 4, 8, 11, 29, 33]
17 : [22, 23, 27, 33, 35, 41]
18 : [2, 5, 10, 30, 31, 34]
19 : [7, 11, 23, 27, 29, 33]
20 : [6, 28, 34, 38, 43, 44]
21 : [6, 17, 20, 21, 42, 44]
22 : [6, 13, 20, 22, 30, 38]
23 : [3, 8, 11, 20, 30, 33]
24 : [2, 7, 26, 30, 31, 42]
25 : [15, 17, 20, 26, 31, 37]
26 : [2, 4, 7, 17, 31, 35]
27 : [10, 14, 27, 33, 37, 41]
28 : [3, 8, 13, 27, 37, 41]
29 : [4, 8, 13, 14, 16, 37]
30 : [9, 14, 17, 20, 28, 44]
31 : [8, 11, 13, 14, 35, 38]
32 : [2, 23, 31, 35, 38, 42]
33 : [1, 4, 5, 33, 38, 43]
34 : [1, 3, 12, 26, 27, 

In [104]:
# 88회부터 지금까지 1등부터 5등까지 상금의 평균낸다.
mean_prize = [ np.mean(rows[87:, 8]),
           np.mean(rows[87:, 9]),
           np.mean(rows[87:, 10]),
           50000, 5000]

print(mean_prize)

train_idx = (0, 800)
val_idx = (801, 900)
test_idx = (901, len(x_samples))

[2437249378.545354, 57206335.696902655, 1450902.4623893804, 50000, 5000]


In [105]:
# 등수와 상금을 반환함
# 순위에 오르지 못한 경우에는 등수가 0으로 반환함
def calc_reward(true_numbers, true_bonus, pred_numbers):

    count = 0

    for ps in pred_numbers:
        if ps in true_numbers:
            count += 1

    if count == 6:
        return 0, mean_prize[0]
    elif count == 5 and true_bonus in pred_numbers:
        return 1, mean_prize[1]
    elif count == 5:
        return 2, mean_prize[2]
    elif count == 4:
        return 3, mean_prize[3]
    elif count == 3:
        return 4, mean_prize[4]

    return 5, 0

In [106]:
train_total_reward = []
train_total_grade = np.zeros(6, dtype=int)

val_total_reward = []
val_total_grade = np.zeros(6, dtype=int)

test_total_reward = []
test_total_grade = np.zeros(6, dtype=int)

model.reset_states()

'''
#지난회차 역수 inverse_numbers에 넣기
inverse_numbers = []
for i in range(6):
    a = 46 - ohbin2numbers(y_samples[-1])[i]
    inverse_numbers.append(a)
inverse_numbers.sort()
'''

print('[No. ] 1st 2nd 3rd 4th 5th 6th Rewards')


for i in range(len(x_samples)):
#    xs = x_samples[i].reshape(1, 1, 45)
    xs = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],-1))

    ys_pred = model.predict_on_batch(xs) # 모델의 출력값을 얻음
    
    
    list_numbers = []
    inverse_numbers = []

    for j in range(6):
        a = 46 - ohbin2numbers(y_samples[-1])[j]
        inverse_numbers.append(a)
    inverse_numbers.sort()

    #여기부터 제외수 만들기
    expect_numbers=[]
    # 확률 하위 16%는 제외수로 날려
    _mean = np.mean(ys_pred[0])
    _std = np.std(ys_pred[0])
    for k in range(45):
        if ys_pred[0][k] <= (_mean - _std):
            expect_numbers.append(k+1)
    
    '''
    #평균*0.9 보다 작은 값은 제외수로 넣어
    _sum = 0
    _mean = 0
    for x in ys_pred[0]:
        if x <= 0:
            x = 0
        else:
            _sum += x
    _mean = _sum / len(ys_pred[0])

    _expect = []

    for k in range(45):
        if ys_pred[0][k] <= (_mean * 0.985):
            expect_numbers.append(k+1)
    '''
    sum_reward = 0
    sum_grade = np.zeros(6, dtype=int) # 6등까지 변수

    for n in range(10): # 10판 수행
        numbers = gen_numbers_from_probability(ys_pred[0],expect_numbers, inverse_numbers)
          
        
        #i회차 입력 후 나온 출력을 i+1회차와 비교함
        grade, reward = calc_reward(rows[i+1,1:7], rows[i+1,7], numbers) 
        
        sum_reward += reward
        sum_grade[grade] += 1

        if i >= train_idx[0] and i < train_idx[1]:
            train_total_grade[grade] += 1
        elif i >= val_idx[0] and i < val_idx[1]:
            val_total_grade[grade] += 1
        elif i >= test_idx[0] and i < test_idx[1]:
            val_total_grade[grade] += 1
    
    if i >= train_idx[0] and i < train_idx[1]:
        train_total_reward.append(sum_reward)
    elif i >= val_idx[0] and i < val_idx[1]:
        val_total_reward.append(sum_reward)
    elif i >= test_idx[0] and i < test_idx[1]:
        test_total_reward.append(sum_reward)
                        
    print('[{0:4d}] {1:3d} {2:3d} {3:3d} {4:3d} {5:3d} {6:3d} {7:15,d}'.format(i+1, sum_grade[0], sum_grade[1], sum_grade[2], sum_grade[3], sum_grade[4], sum_grade[5], int(sum_reward)))

print('Total') 
print('==========')    
print('Train {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(train_total_grade[0], train_total_grade[1], train_total_grade[2], train_total_grade[3], train_total_grade[4], train_total_grade[5], int(sum(train_total_reward))))
print('Val   {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(val_total_grade[0], val_total_grade[1], val_total_grade[2], val_total_grade[3], val_total_grade[4], val_total_grade[5], int(sum(val_total_reward))))
print('Test  {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(test_total_grade[0], test_total_grade[1], test_total_grade[2], test_total_grade[3], test_total_grade[4], test_total_grade[5], int(sum(test_total_reward))))
print('==========')


[No. ] 1st 2nd 3rd 4th 5th 6th Rewards
[   1]   0   0   0   0   1   9           5,000
[   2]   0   0   0   0   0  10               0
[   3]   0   0   0   0   0  10               0
[   4]   0   0   0   0   1   9           5,000
[   5]   0   0   0   0   0  10               0
[   6]   0   0   0   0   0  10               0
[   7]   0   0   0   0   0  10               0
[   8]   0   0   0   0   0  10               0
[   9]   0   0   0   0   0  10               0
[  10]   0   0   0   0   0  10               0
[  11]   0   0   0   0   0  10               0
[  12]   0   0   0   0   0  10               0
[  13]   0   0   0   0   3   7          15,000
[  14]   0   0   0   0   0  10               0
[  15]   0   0   0   0   0  10               0
[  16]   0   0   0   0   0  10               0
[  17]   0   0   0   0   1   9           5,000
[  18]   0   0   0   0   0  10               0
[  19]   0   0   0   0   0  10               0
[  20]   0   0   0   0   0  10               0
[  21]   0   0   0   

[ 176]   0   0   0   0   1   9           5,000
[ 177]   0   0   0   0   0  10               0
[ 178]   0   0   0   0   0  10               0
[ 179]   0   0   0   0   1   9           5,000
[ 180]   0   0   0   0   0  10               0
[ 181]   0   0   0   0   1   9           5,000
[ 182]   0   0   0   0   0  10               0
[ 183]   0   0   0   0   1   9           5,000
[ 184]   0   0   0   0   0  10               0
[ 185]   0   0   0   0   1   9           5,000
[ 186]   0   0   0   0   0  10               0
[ 187]   0   0   0   0   1   9           5,000
[ 188]   0   0   0   0   0  10               0
[ 189]   0   0   0   0   0  10               0
[ 190]   0   0   0   0   0  10               0
[ 191]   0   0   0   0   0  10               0
[ 192]   0   0   0   0   0  10               0
[ 193]   0   0   0   0   0  10               0
[ 194]   0   0   0   0   1   9           5,000
[ 195]   0   0   0   0   0  10               0
[ 196]   0   0   0   0   0  10               0
[ 197]   0   

[ 352]   0   0   0   0   0  10               0
[ 353]   0   0   0   0   0  10               0
[ 354]   0   0   0   0   1   9           5,000
[ 355]   0   0   0   0   0  10               0
[ 356]   0   0   0   0   1   9           5,000
[ 357]   0   0   0   0   0  10               0
[ 358]   0   0   0   0   0  10               0
[ 359]   0   0   0   0   0  10               0
[ 360]   0   0   0   0   0  10               0
[ 361]   0   0   0   0   0  10               0
[ 362]   0   0   0   0   0  10               0
[ 363]   0   0   0   0   0  10               0
[ 364]   0   0   0   0   0  10               0
[ 365]   0   0   0   0   0  10               0
[ 366]   0   0   0   0   0  10               0
[ 367]   0   0   0   0   1   9           5,000
[ 368]   0   0   0   0   0  10               0
[ 369]   0   0   0   0   0  10               0
[ 370]   0   0   0   0   0  10               0
[ 371]   0   0   0   0   0  10               0
[ 372]   0   0   0   0   0  10               0
[ 373]   0   

[ 527]   0   0   0   0   0  10               0
[ 528]   0   0   0   0   0  10               0
[ 529]   0   0   0   0   1   9           5,000
[ 530]   0   0   0   0   0  10               0
[ 531]   0   0   0   0   0  10               0
[ 532]   0   0   0   0   0  10               0
[ 533]   0   0   0   0   0  10               0
[ 534]   0   0   0   0   0  10               0
[ 535]   0   0   0   0   1   9           5,000
[ 536]   0   0   0   0   0  10               0
[ 537]   0   0   0   0   0  10               0
[ 538]   0   0   0   0   0  10               0
[ 539]   0   0   0   0   0  10               0
[ 540]   0   0   0   0   0  10               0
[ 541]   0   0   0   0   0  10               0
[ 542]   0   0   0   0   0  10               0
[ 543]   0   0   0   0   0  10               0
[ 544]   0   0   0   0   1   9           5,000
[ 545]   0   0   0   0   0  10               0
[ 546]   0   0   0   0   2   8          10,000
[ 547]   0   0   0   0   0  10               0
[ 548]   0   

[ 702]   0   0   0   0   1   9           5,000
[ 703]   0   0   0   0   2   8          10,000
[ 704]   0   0   0   0   0  10               0
[ 705]   0   0   0   0   0  10               0
[ 706]   0   0   0   0   1   9           5,000
[ 707]   0   0   0   0   0  10               0
[ 708]   0   0   0   0   0  10               0
[ 709]   0   0   0   0   1   9           5,000
[ 710]   0   0   0   0   0  10               0
[ 711]   0   0   0   0   0  10               0
[ 712]   0   0   0   0   0  10               0
[ 713]   0   0   0   0   1   9           5,000
[ 714]   0   0   0   0   1   9           5,000
[ 715]   0   0   0   0   0  10               0
[ 716]   0   0   0   0   0  10               0
[ 717]   0   0   0   0   0  10               0
[ 718]   0   0   0   0   0  10               0
[ 719]   0   0   0   0   0  10               0
[ 720]   0   0   0   0   0  10               0
[ 721]   0   0   0   0   0  10               0
[ 722]   0   0   0   0   0  10               0
[ 723]   0   

[ 878]   0   0   0   0   1   9           5,000
[ 879]   0   0   0   0   0  10               0
[ 880]   0   0   0   0   0  10               0
[ 881]   0   0   0   0   0  10               0
[ 882]   0   0   0   0   1   9           5,000
[ 883]   0   0   0   0   0  10               0
[ 884]   0   0   0   0   0  10               0
[ 885]   0   0   0   0   0  10               0
[ 886]   0   0   0   0   0  10               0
[ 887]   0   0   0   0   0  10               0
[ 888]   0   0   0   0   0  10               0
[ 889]   0   0   0   0   0  10               0
[ 890]   0   0   0   0   1   9           5,000
[ 891]   0   0   0   0   0  10               0
[ 892]   0   0   0   1   0   9          50,000
[ 893]   0   0   0   0   0  10               0
[ 894]   0   0   0   0   0  10               0
[ 895]   0   0   0   0   0  10               0
[ 896]   0   0   0   0   0  10               0
[ 897]   0   0   0   0   0  10               0
[ 898]   0   0   0   0   1   9           5,000
[ 899]   0   

In [101]:
ys_pred[0]

array([0.12512231, 0.12561676, 0.12479052, 0.12605903, 0.12599432,
       0.12599441, 0.12457421, 0.12579933, 0.12516183, 0.12478355,
       0.12432635, 0.12501019, 0.12421331, 0.12458509, 0.12467206,
       0.12530741, 0.1253146 , 0.        , 0.12453452, 0.12588462,
       0.12503833, 0.12437192, 0.12677607, 0.        , 0.        ,
       0.12420872, 0.1262312 , 0.12495261, 0.1243636 , 0.12490141,
       0.12468022, 0.12550631, 0.12597463, 0.12483609, 0.12513277,
       0.        , 0.12459087, 0.12504187, 0.        , 0.        ,
       0.12577897, 0.12450883, 0.12478215, 0.12515572, 0.        ],
      dtype=float32)

In [102]:
expect_numbers.sort()
expect_numbers

[18, 24, 25, 25, 36, 39, 40, 45]

In [103]:
np.argmax(ys_pred[0])+1

23